# About

It's difficult to use functions in jupyter notebook, since we want different steps to be in different cells, so one of the main functions of this module is to emulate a function like scope of the variables - which get destroyed at the end of the experiment. Some extra magic is added to reclaim GPU and General RAM.

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from ipyexperiments import *

## Setup && Preload

Prep.

In [3]:
import numpy as np
import torch

In [4]:
def consume_gpu(n): return torch.ones((n, n)).cuda()
def consume_cpu(n): return np.ones((n, n))

`pytorch` seems to latch on ~0.5GB GPU RAM, and ~2GB of RAM upon its first use, so let's pre-load it, so that the experiments' stats are not misleading.

In [5]:
z = consume_gpu(1)

## Experiment 1: Consume General RAM

Let's consume a big chunk of non-GPU RAM and reclaim it at the end of the experiment

In [6]:
exp1 = IPyExperiments() # consume some cpu ram

Starting experiment...

*** Current state:
Gen RAM Free 11.8 GB | Proc size 2.1 GB
GPU RAM Free  7.5 GB | Used 1.0 GB | Util 12.0% | Total 8.5 GB


In [7]:
x1 = consume_cpu(2**14) # about 1.5GB

In [8]:
x2 = consume_cpu(2**14) # about 1.5GB

In [9]:
del exp1 # finish experiment

Finishing experiment...

*** Deleting the following local variables:
['x2', 'x1']

*** RAM consumed during the experiment:
Gen: 4.3 GB
GPU: 0 Bytes

*** RAM reclaimed at the end of the experiment:
Gen: 4.3 GB (100.00%)
GPU: 0 Bytes (100.00%)

*** Current state:
Gen RAM Free 11.8 GB | Proc size 2.1 GB
GPU RAM Free  7.5 GB | Used 1.0 GB | Util 12.0% | Total 8.5 GB


## Experiment 2: Consume General and GPU RAM

Let's consume a big chunk of non-GPU and GPU RAM and reclaim both at the end of the experiment

In [10]:
exp2 = IPyExperiments() # consume some gpu and cpu ram

Starting experiment...

*** Current state:
Gen RAM Free 11.8 GB | Proc size 2.1 GB
GPU RAM Free  7.5 GB | Used 1.0 GB | Util 12.0% | Total 8.5 GB


In [11]:
x1 = consume_cpu(2**14) # about 1.5GB

In [12]:
x2 = consume_gpu(2**14) # about 1GB

In [13]:
del exp2 # finish experiment

Finishing experiment...

*** Deleting the following local variables:
['x2', 'x1']

*** RAM consumed during the experiment:
Gen: 2.1 GB
GPU: 1.1 GB

*** RAM reclaimed at the end of the experiment:
Gen: 2.1 GB (100.00%)
GPU: 1.1 GB (100.00%)

*** Current state:
Gen RAM Free 11.8 GB | Proc size 2.1 GB
GPU RAM Free  7.5 GB | Used 1.0 GB | Util 12.0% | Total 8.5 GB


## Experiment 3: Get Stats Data

Here we demonstate features that help with using this framework programmatically. i.e. getting the functions to return stats during and at the end of the experiment, rather than just printing it. You can then use it to programmatically refine the parameters before rerunning the experiment.

In [14]:
exp3 = IPyExperiments() # consume some gpu and cpu ram

Starting experiment...

*** Current state:
Gen RAM Free 11.8 GB | Proc size 2.1 GB
GPU RAM Free  7.5 GB | Used 1.0 GB | Util 12.0% | Total 8.5 GB


In [15]:
x1 = consume_cpu(2**14) # about 1.5GB

Run an intermediary report of how much of the resources was consumed, and how much is available, returning the data as numbers. (none would be reclaimed yet, so it'll be zeros, but the return value is there for consistency).

In [16]:
consumed, reclaimed, available = exp3.get_stats()
print(consumed, reclaimed, available)

{'gen_ram': 2147504128, 'gpu_ram': 0} {'gen_ram': 0, 'gpu_ram': 0} {'gen_ram': 9627705344, 'gpu_ram': 7487881216}


In [17]:
x2 = consume_gpu(2**14) # about 1GB

Run another intermediary report.

In [18]:
consumed, reclaimed, available = exp3.get_stats()
print(consumed, reclaimed, available)

{'gen_ram': 2147528704, 'gpu_ram': 1073741824} {'gen_ram': 0, 'gpu_ram': 0} {'gen_ram': 9625780224, 'gpu_ram': 6414139392}


Run the final report of how much of the resources was consumed, and how much is available, and how much was reclaimed, returning the data as numbers.

In [19]:
final_consumed, final_reclaimed, final_available = exp3.finish() # finish experiment
print("\nNumerical data:\n", final_consumed, final_reclaimed, final_available)

Finishing experiment...

*** Deleting the following local variables:
['reclaimed', 'x1', 'consumed', 'available', 'exp3', 'x2']

*** RAM consumed during the experiment:
Gen: 2.1 GB
GPU: 1.1 GB

*** RAM reclaimed at the end of the experiment:
Gen: 2.1 GB (100.00%)
GPU: 1.1 GB (100.00%)

*** Current state:
Gen RAM Free 11.8 GB | Proc size 2.1 GB
GPU RAM Free  7.5 GB | Used 1.0 GB | Util 12.0% | Total 8.5 GB

Numerical data:
 {'gen_ram': 2147532800, 'gpu_ram': 1073741824} {'gen_ram': 2147483648, 'gpu_ram': 1073741824} {'gen_ram': 11776757760, 'gpu_ram': 7487881216}
